In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import warnings
import json
import plotly.express as px
from urllib.request import urlopen
import seaborn as sns

In [2]:
complete_df = pd.read_csv('complete.csv')
print(complete_df.shape)
complete_df.columns

(2552, 18)


Index(['Address', 'Count', 'Zipcode', 'Type', 'Owner Occ?', 'Purchase Amt',
       'Amount Mortgaged', '% Borrowed', 'Mtg Tx Pd', 'Mansion Tax Pd',
       'New Sales Tax Amount', 'Population', 'Median Age', 'Household Income',
       'Per Capita Income', 'Median Gross Rent', 'Poverty Count',
       'Poverty Rate'],
      dtype='object')

In [ ]:
# complete_filtered = pd.DataFrame(complete_df[['Purchase Amt',
#        'Amount Mortgaged', '% Borrowed', 'Mtg Tx Pd',
#        New Sales Tax Amount']])
# filtered_columns = ['Purchase Amt',
#        'Amount Mortgaged', '% Borrowed', 'Mtg Tx Pd',
#        'New Sales Tax Amount']

# # for col in filtered_columns:
# #     print(complete_filtered[col].describe())

In [3]:
#quick filter of data to show that the top 5 mtg amounts show a 
#135% increase over the 6th top mtg amount
complete_df = complete_df.sort_values('Mtg Tx Pd', ascending=False)
complete_df.head()


,Address,Count,Zipcode,Type,Owner Occ?,Purchase Amt,Amount Mortgaged,% Borrowed,Mtg Tx Pd,Mansion Tax Pd,New Sales Tax Amount,Population,Median Age,Household Income,Per Capita Income,Median Gross Rent,Poverty Count,Poverty Rate
1333,464 GREENWICH ST,1,10013,SFR,1.0,23500000.0,14100000.0,60.000000,274950.00,235000.00,1.042930e+06,29453.0,40.2,150675.0,157378.0,2113.0,2762.0,9.377653
809,30 PARK PL PH78B,1,10007,CND,1.0,18439344.0,11862500.0,64.332549,231318.75,184393.44,8.183381e+05,7506.0,34.7,250001.0,191709.0,3501.0,152.0,2.025047
1404,27 VANDAM ST,1,10013,SFR,1.0,14800000.0,11840000.0,80.000000,230880.00,148000.00,6.568240e+05,29453.0,40.2,150675.0,157378.0,2113.0,2762.0,9.377653
1236,7 HUBERT ST PHC,1,10013,CND,1.0,15000000.0,10000000.0,66.666667,195000.00,150000.00,6.657000e+05,29453.0,40.2,150675.0,157378.0,2113.0,2762.0,9.377653
2214,24 LEONARD ST 4,1,10013,CND,1.0,14700000.0,8880000.0,60.408163,173160.00,147000.00,6.523860e+05,29453.0,40.2,150675.0,157378.0,2113.0,2762.0,9.377653


In [4]:
complete_df.describe()

,Count,Zipcode,Owner Occ?,Purchase Amt,Amount Mortgaged,% Borrowed,Mtg Tx Pd,Mansion Tax Pd,New Sales Tax Amount,Population,Median Age,Household Income,Per Capita Income,Median Gross Rent,Poverty Count,Poverty Rate
count,2552.0,2552.000000,2552.000000,2.552000e+03,2.552000e+03,2552.000000,2552.000000,2552.000000,2.552000e+03,2552.000000,2552.000000,2552.000000,2552.000000,2552.000000,2552.000000,2552.000000
mean,1.0,10621.760580,0.711207,2.088964e+06,7.062190e+05,35.514634,13661.602573,17904.026050,7.361444e+04,49828.514890,40.701528,126074.266850,99758.574843,2257.229232,9130.142241,15.914333
std,0.0,649.914836,0.453290,2.571257e+06,1.194865e+06,33.996108,23332.138229,27503.198626,1.049558e+05,36734.675922,4.518679,71909.451007,68885.614785,726.184283,8151.095669,7.746927
min,1.0,10007.000000,0.000000,3.500000e+04,0.000000e+00,0.000000,0.000000,0.000000,1.540000e+03,6259.000000,30.800000,34800.000000,19016.000000,1362.000000,152.000000,2.025047
25%,1.0,10013.000000,0.000000,6.100000e+05,0.000000e+00,0.000000,0.000000,0.000000,1.819580e+04,7506.000000,38.700000,52658.000000,28128.000000,1625.000000,1493.000000,9.377653
50%,1.0,10069.000000,1.000000,1.100000e+06,3.000000e+05,40.778534,5400.000000,11000.000000,3.679102e+04,29453.000000,40.200000,150675.000000,148668.000000,2113.000000,2762.000000,20.452859
75%,1.0,11355.000000,1.000000,2.900000e+06,7.762500e+05,68.854283,15136.875000,29000.000000,9.916988e+04,79495.000000,46.200000,197680.000000,157378.000000,3286.000000,16259.000000,20.452859
max,1.0,11368.000000,1.000000,4.750000e+07,1.410000e+07,98.738462,274950.000000,475000.000000,2.108050e+06,112750.000000,46.200000,250001.000000,191709.000000,3501.000000,27796.000000,34.578591


In [ ]:

# for col_name in filtered_columns:
#     quartiles = complete_filtered[col_name].quantile([.25, .5, .75])
#     lowerq= quartiles[0.25]
#     upperq = quartiles[0.75]
#     iqr = upperq - lowerq
#     lower_bound = lowerq - (1.5*iqr)
#     upper_bound = upperq + (1.5* iqr)
#     outliers =complete_filtered[(complete_filtered[col_name] < lower_bound) | (complete_filtered[col_name] > upper_bound)]
#     print(f"{col_name}'s potentional outliers: {outliers}")



In [ ]:
# fig, axes = plt.subplots(nrows=len(filtered_columns), ncols=1, figsize=(8, 2 * len(filtered_columns)))


# for i, column in enumerate(filtered_columns):
#     axes[i].boxplot(complete_filtered[column])
#     axes[i].set_title(f'Boxplot: {column}')
#     axes[i].set_ylabel(column)
    
# plt.tight_layout()
# plt.show()

In [ ]:
# new_filt = complete_df[[ 'Type', 'Owner Occ?', 'Purchase Amt',
#        'Amount Mortgaged', '% Borrowed', 'Mtg Tx Pd',
#        'New Sales Tax Amount', 'Population', 'Median Age', 'Household Income',
#        'Median Gross Rent']]

# col_to_plot = ['Purchase Amt',
#        'Amount Mortgaged', '% Borrowed', 'Mtg Tx Pd',
#        'New Sales Tax Amount', 'Population', 'Household Income', 'Median Gross Rent']


In [ ]:
col_to_plot = []

def plotscat(col_to_plot, df):
    fig, axes = plt.subplots(nrows=len(col_to_plot), ncols=1, figsize=(8, 2 * len(col_to_plot)))
    for i, column in enumerate(col_to_plot):
        axes[i].scatter(df['Household Income'], df[column])
        axes[i].set_title(f'Scatter Plot: Household Income vs {column}')
        axes[i].set_xlabel('Household Income')
        axes[i].set_ylabel(column)
        plt.tight_layout()
        plt.show()

In [5]:
#NEED TO REWRITE THIS - BOOLEAN
complete_df.drop('Type', axis=1, inplace=True)

In [6]:
#create new columns for % borrowed, total taxes paid and taxes as a percent of household income and taxes as a percent of gross rent

#divide monetary amounts by 1000 to make the units per 1000
complete_df['Zip'] = complete_df['Zipcode'].astype(str)
complete_df['Tot tax pd'] = complete_df['Mtg Tx Pd'] + complete_df['Mansion Tax Pd']
complete_df['Mtax as % Household Income'] = complete_df['Mtg Tx Pd']/complete_df['Household Income']* 100
complete_df['Months rent spent on Mtax'] = complete_df['Median Gross Rent']/complete_df['Mtg Tx Pd']



In [7]:
complete_df["Purchase Amt"] = complete_df["Purchase Amt"] / 1000
complete_df["Amount Mortgaged"] = complete_df["Amount Mortgaged"] / 1000
complete_df["Mtg Tx Pd"] = complete_df["Mtg Tx Pd"] / 1000
complete_df["Mansion Tax Pd"] = complete_df["Mansion Tax Pd"] / 1000
complete_df["New Sales Tax Amount"] = complete_df["New Sales Tax Amount"] / 1000
complete_df["Per Capita Income"] = complete_df["Per Capita Income"] / 1000

In [8]:
complete_df.shape

(2552, 21)

In [12]:
#Seperate the data into 4 dataframes, by areas and status

bklyn_df = complete_df.loc[~complete_df['Zipcode'].isin([10013, 10007, 10282, 10453, 11355, 10457, 11368])]
bklyn_investors = bklyn_df[(bklyn_df['Owner Occ?'] ==1)]
bklyn_oocc = bklyn_df[(bklyn_df['Owner Occ?'] ==0)]



(1520, 21)

In [10]:
bklyn_oocc.shape

(95, 21)

In [11]:
fourframes= [bklyn_oocc, bklyn_investors, high_low_oocc, high_low_investors]

column_load = {'Count': 'sum', "Population": 'mean', 'Count': 'sum', 'Purchase Amt': 'mean',
            '% Borrowed': 'mean',
           'Amount Mortgaged': 'mean','Mtg Tx Pd': 'mean', 'Mtax as % Household Income': 'mean', 
            'Months rent spent on Mtax': 'mean',
           'Mansion Tax Pd': 'mean', 'New Sales Tax Amount': 'mean', 
           'Per Capita Income': 'mean', 'Household Income': 'mean'}


In [ ]:
bklyn_investors_by_zip = pd.DataFrame(data= bklyn_investors)
bklyn_investors_by_zip = bklyn_investors_by_zip.groupby('Zip').agg(column_load)
#bklyn_investors_by_zip = bklyn_investors_by_zip.applymap("{:.2f}".format)
bklyn_investors_by_zip.head()

In [ ]:
bklyn_oocc_by_zip = pd.DataFrame(bklyn_oocc)
bklyn_oocc_by_zip = bklyn_oocc_by_zip.groupby(['Zip'], axis=0).agg(column_load)
#bklyn_oocc_by_zip = bklyn_oocc_by_zip.applymap("{:.2f}".format)
bklyn_oocc_by_zip.head()

In [ ]:
bklyn_oocc_by_zip.describe()

In [ ]:
bklyn_investors_by_zip[['Count', 'Population', 'Purchase Amt', '% Borrowed',
       'Amount Mortgaged', 'Mtg Tx Pd', 'Mansion Tax Pd',
       'New Sales Tax Amount', 'Per Capita Income', 'Household Income']]=bklyn_investors_by_zip[['Count', 'Population', 'Purchase Amt', '% Borrowed',
       'Amount Mortgaged', 'Mtg Tx Pd', 'Mansion Tax Pd',
       'New Sales Tax Amount', 'Per Capita Income', 'Household Income']].astype('float64')
bklyn_investors_by_zip.sort_values(by=['Household Income'], inplace=True)
bklyn_investors_by_zip.head()

In [ ]:
bklyn_oocc_by_zip[['Count', 'Population', 'Purchase Amt', '% Borrowed',
       'Amount Mortgaged', 'Mtg Tx Pd', 'Mansion Tax Pd',
       'New Sales Tax Amount', 'Per Capita Income', 'Household Income']]=bklyn_oocc_by_zip[['Count', 'Population', 'Purchase Amt', '% Borrowed',
       'Amount Mortgaged', 'Mtg Tx Pd', 'Mansion Tax Pd',
       'New Sales Tax Amount', 'Per Capita Income', 'Household Income']].astype('float64')
bklyn_oocc_by_zip.sort_values(by=['Household Income'], inplace=True)
bklyn_oocc_by_zip.head(20)

In [ ]:
bklyn_oocc_by_zip['mtg %'] = bklyn_oocc_by_zip['Amount Mortgaged']/bklyn_oocc_by_zip['Purchase Amt']*100
bklyn_investors_by_zip['mtg %'] = bklyn_investors_by_zip['Amount Mortgaged']/bklyn_oocc_by_zip['Purchase Amt']*100
bklyn_oocc_by_zip.head()


In [ ]:
col_to_plot = ['Purchase Amt','% Borrowed', 'Amount Mortgaged',
       'Mtg Tx Pd', 'Mtax as % Household Income', 'Months rent spent on Mtax',
       'New Sales Tax Amount',
       'mtg %']

def plotscat(col_to_plot, df):
    fig, axes = plt.subplots(nrows=len(col_to_plot), ncols=1, figsize=(8, 2 * len(col_to_plot)))
    for i, column in enumerate(col_to_plot):
        axes[i].scatter(df['Household Income'], df[column])
        axes[i].set_title(f'Scatter Plot: Household Income vs {column}')
        axes[i].set_xlabel('Household Income')
        axes[i].set_ylabel(column)
    plt.tight_layout()
    plt.show()

In [ ]:
plotscat(col_to_plot, bklyn_oocc_by_zip)

In [ ]:
bklyn_oocc_by_zip.shape

In [ ]:

plt.figure(figsize=(15,5))



plt.plot(bklyn_oocc_by_zip['Household Income'], bklyn_oocc_by_zip['% Borrowed'], marker="x", label='Bklyn Residence Purchase')
plt.plot(bklyn_oocc_by_zip['Household Income'], bklyn_investors_by_zip['% Borrowed'], marker="o", label='Bklyn Investment Purchase')
plt.yticks(fontsize=15)
plt.xlabel('Zipcode sorted by Household Income (in thousands)', fontsize=20)
plt.ylabel('Percent Borrowed', fontsize=20)
plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(high_low_oocc_by_zip['Household Income'], high_low_oocc_by_zip['% Borrowed'], marker="x", label='high_low Residence Purchase')
plt.plot(high_low_oocc_by_zip['Household Income'], high_low_investors_by_zip['% Borrowed'], marker="o", label='high_low Investment Purchase')
#plt.plot(manhatt_oocc_df['Per Capita Income'], investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
#plt.xticks(high_low_oocc_df['Per Capita Income'], rotation=90)
#plt.xticks(), fontsize=15, rotation='vertical')
plt.yticks(fontsize=15)
plt.xlabel('Zipcode sorted by Household Income (in thousands)', fontsize=20)
plt.ylabel('Percent Borrowed', fontsize=20)
plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(high_low_oocc_by_zip['Per Capita Income'], yaxis, marker="x", label='Bklyn Residence Purchase')
plt.plot(high_low_oocc_by_zip['Per Capita Income'], high_low_investors_by_zip['% borrowed'], marker="o", label='Bklyn Investment Purchase')
    #plt.plot(manhatt_oocc_df['Per Capita Income'], investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
    #plt.xticks(bklyn_oocc_df['Per Capita Income'], rotation=90)
    #plt.xticks(), fontsize=15, rotation='vertical')
plt.yticks(fontsize=15)
plt.xlabel('Zipcode sorted by Per Capita Income (in thousands)', fontsize=20)
plt.ylabel('Percent Borrowed', fontsize=20)
plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in other 8 2022', fontsize=20)
plt.legend()
plt.show()

In [ ]:
bklyn_sorted = bklyn_oocc_by_zip.sort_values('Per Capita Income')
bklyn_sorted.head()

# bklyn_oocc_by_zip.sort_values(['Per Capita Income'], inplace=True)
# bklyn_oocc_by_zip

In [ ]:
#make a chart of the % borrowed for the zip in ascending order but first sort dfs in order

#fig = plt.figure()
#ax1 = fig.add_subplot()



bklyn_sorted.plot('Per Capita Income', '% borrowed')
plt.show()

#bklyn_oocc_by_zip.set_index(['Per Capita Income'), inplace = True)
#bklyn_oocc_by_zip.sort_index()
#bklyn_investors_by_zip.sort_values('Per Capita Income', inplace = True)
#bklyn_investors_by_zip.sort_index()
#bklyn_oocc_df['perCI'] = bklyn_oocc_df.index.astype(str)
#manhattan_oocc_df.set_index('Per Capita Income', inplace = True)
# #manhattan_investor_df.set_index('Per Capita Income', inplace = True)

# graphaxis1 = pd.DataFrame(bklyn_oocc_by_zip['Per Capita Income'], ['% borrowed'])

# plt.figure(figsize=(15,10))



# plt.plot(graphaxis1['Per Capita Income'], graphaxis1['% borrowed'], marker="x", label='Bklyn Residence Purchase')
# #plt.plot(x = bklyn_investors_by_zip['Per Capita Income'], y =bklyn_investors_by_zip['% borrowed'], marker="o", label='Bklyn Investment Purchase')
# #plt.plot(manhatt_oocc_df['Per Capita Income'], investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
# #plt.xticks(bklyn_oocc_df['Per Capita Income'], rotation=90)
# plt.xticks(['Per Capita Income'], fontsize=15, rotation='vertical')
# plt.yticks(fontsize=15)
# plt.xlabel('Zipcode sorted by Per Capita Income (in thousands)', fontsize=20)
# plt.ylabel('Percent Borrowed', fontsize=20)
# plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
# plt.legend()
# plt.show()

In [ ]:
# bklyn_oocc = bklyn_oocc.groupby(['Zipcode']).agg({'Count': 'sum', 'Purchase Amt': 'mean', #                                                             'Purchase Down %': 'mean', '% borrowed': 'mean',
#                                                             'Amount Mortgaged': 'mean', 
#                                                             'Mtg Tx Pd': 'mean', 'Mansion Tax Pd': 'mean', 
#                                                             'New Sales Tax Amount': 'mean', 
#                                                              'Per Capita Income': 'mean'})

# bklyn_oocc = bklyn_oocc.applymap("{:.2f}".format)
# bklyn_oocc.head()

In [ ]:
high_low_oocc = high_low_oocc.groupby(['Zipcode']).agg({'Zipcode': 'Zipcode', "Population": 'mean', 'Count': 'sum', 'Purchase Amt': 'mean',
           'Purchase Down %': 'mean', '% borrowed': 'mean',
           'Amount Mortgaged': 'mean','Mtg Tx Pd': 'mean',
           'Mansion Tax Pd': 'mean', 'New Sales Tax Amount': 'mean', 
           'Per Capita Income': 'mean'})

high_low_oocc = high_low_oocc.applymap("{:.2f}".format)
high_low_oocc.head()

In [ ]:
#bklyn_oocc.set_index('Per Capita Income', inplace = True)
bklyn_investors.set_index('Per Capita Income', inplace = True)
# bklyn_oocc['perCI'] = bklyn_oocc.index.astype(str)
# # manhattan_oocc_df.set_index('Per Capita Income', inplace = True)
# # manhattan_investor_df.set_index('Per Capita Income', inplace = True)


# plt.figure(figsize=(15,5))



plt.plot(bklyn_oocc['Per Capita Income'], bklyn_oocc['percent borrowed'], marker="x", label='Bklyn Residence Purchase')
plt.plot(bklyn_oocc['perCI'], bklyn_investors['percent borrowed'], marker="o", label='Bklyn Investment Purchase')
# #plt.plot(manhatt_oocc_df['Per Capita Income'], investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
# #plt.xticks(bklyn_oocc_df['Per Capita Income'], rotation=90)
# plt.xticks(range(len(bklyn_oocc['Zip'])), fontsize=15, rotation='vertical')
# plt.yticks(fontsize=15)
# plt.xlabel('Zipcode sorted by Per Capita Income (in thousands)', fontsize=20)
# plt.ylabel('Percent Borrowed', fontsize=20)
# plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
# plt.legend()
# plt.show()

bklyn_oocc.head()

In [ ]:
#the next few cells seperate the data into 4 data frames, first by owner 
2 for brooklyn, and two for the other zips.
#each of these are then sp


owner_occ_by_zip = owner_occ_df.groupby(['Zipcode']).agg({'Count': 'sum', 'Purchase Amt': 'mean', 
                                                            'Purchase Down %': 'mean',
                                                            'Amount Mortgaged': 'mean', 
                                                            'Mtg Tx Pd': 'mean', 'Mansion Tax Pd': 'mean', 
                                                            'New Sales Tax Amount': 'mean', 
                                                            'Per Capita Income': 'mean'})


owner_occ_by_zip["Purchase Amt"] = owner_occ_by_zip["Purchase Amt"] / 1000
owner_occ_by_zip["Amount Mortgaged"] = owner_occ_by_zip["Amount Mortgaged"] / 1000
owner_occ_by_zip["Mtg Tx Pd"] = owner_occ_by_zip["Mtg Tx Pd"] / 1000
owner_occ_by_zip["Mansion Tax Pd"] = owner_occ_by_zip["Mansion Tax Pd"] / 1000
owner_occ_by_zip["New Sales Tax Amount"] = owner_occ_by_zip["New Sales Tax Amount"] / 1000
owner_occ_by_zip["Per Capita Income"] = owner_occ_by_zip["Per Capita Income"] / 1000

owner_occ_by_zip["Purchase Amt"] = owner_occ_by_zip["Purchase Amt"].map("{:.2f}".format)
owner_occ_by_zip["Purchase Down %"] = owner_occ_by_zip["Purchase Down %"].map("{:.2f}".format)
owner_occ_by_zip["Amount Mortgaged"] = owner_occ_by_zip["Amount Mortgaged"].map("{:.2f}".format)
owner_occ_by_zip["Mtg Tx Pd"] = owner_occ_by_zip["Mtg Tx Pd"].map("{:.2f}".format)
owner_occ_by_zip["Mansion Tax Pd"] = owner_occ_by_zip["Mansion Tax Pd"].map("{:.2f}".format)
owner_occ_by_zip["New Sales Tax Amount"] = owner_occ_by_zip["New Sales Tax Amount"].map("{:.2f}".format)
owner_occ_by_zip["Per Capita Income"] = owner_occ_by_zip["Per Capita Income"].map("{:.2f}".format)

owner_occ_by_zip["Purchase Amt"] = owner_occ_by_zip["Purchase Amt"].astype("float")
owner_occ_by_zip["Purchase Down %"] = owner_occ_by_zip["Purchase Down %"].astype("float")
owner_occ_by_zip["Amount Mortgaged"] = owner_occ_by_zip["Amount Mortgaged"].astype("float")
owner_occ_by_zip["Mtg Tx Pd"] = owner_occ_by_zip["Mtg Tx Pd"].astype("float")
owner_occ_by_zip["Mansion Tax Pd"] = owner_occ_by_zip["Mansion Tax Pd"].astype("float")
owner_occ_by_zip["New Sales Tax Amount"] = owner_occ_by_zip["New Sales Tax Amount"].astype("float")
owner_occ_by_zip["Per Capita Income"] = owner_occ_by_zip["Per Capita Income"].astype("float")

In [ ]:
investor_by_zip = investor_df.groupby(['Zipcode']).agg({'Count': 'sum', 'Purchase Amt': 'mean', 
                                                            'Purchase Down %': 'mean',
                                                            'Amount Mortgaged': 'mean', 
                                                            'Mtg Tx Pd': 'mean', 'Mansion Tax Pd': 'mean', 
                                                            'New Sales Tax Amount': 'mean', 
                                                            'Per Capita Income': 'mean'})


investor_by_zip["Purchase Amt"] = investor_by_zip["Purchase Amt"] / 1000
investor_by_zip["Amount Mortgaged"] = investor_by_zip["Amount Mortgaged"] / 1000
investor_by_zip["Mtg Tx Pd"] = investor_by_zip["Mtg Tx Pd"] / 1000
investor_by_zip["Mansion Tax Pd"] = investor_by_zip["Mansion Tax Pd"] / 1000
investor_by_zip["New Sales Tax Amount"] = investor_by_zip["New Sales Tax Amount"] / 1000
investor_by_zip["Per Capita Income"] = investor_by_zip["Per Capita Income"] / 1000

investor_by_zip["Purchase Amt"] = investor_by_zip["Purchase Amt"].map("{:.2f}".format)
investor_by_zip["Purchase Down %"] = investor_by_zip["Purchase Down %"].map("{:.2f}".format)
investor_by_zip["Amount Mortgaged"] = investor_by_zip["Amount Mortgaged"].map("{:.2f}".format)
investor_by_zip["Mtg Tx Pd"] = investor_by_zip["Mtg Tx Pd"].map("{:.2f}".format)
investor_by_zip["Mansion Tax Pd"] = investor_by_zip["Mansion Tax Pd"].map("{:.2f}".format)
investor_by_zip["New Sales Tax Amount"] = investor_by_zip["New Sales Tax Amount"].map("{:.2f}".format)
investor_by_zip["Per Capita Income"] = investor_by_zip["Per Capita Income"].map("{:.2f}".format)

investor_by_zip["Purchase Amt"] = investor_by_zip["Purchase Amt"].astype("float")
investor_by_zip["Purchase Down %"] = investor_by_zip["Purchase Down %"].astype("float")
investor_by_zip["Amount Mortgaged"] = investor_by_zip["Amount Mortgaged"].astype("float")
investor_by_zip["Mtg Tx Pd"] = investor_by_zip["Mtg Tx Pd"].astype("float")
investor_by_zip["Mansion Tax Pd"] = investor_by_zip["Mansion Tax Pd"].astype("float")
investor_by_zip["New Sales Tax Amount"] = investor_by_zip["New Sales Tax Amount"].astype("float")
investor_by_zip["Per Capita Income"] = investor_by_zip["Per Capita Income"].astype("float")

In [ ]:
owner_occ_by_zip["total_tax_paid"] = owner_occ_by_zip["Mtg Tx Pd"] + owner_occ_by_zip["Mansion Tax Pd"]
owner_occ_by_zip['percent borrowed'] = 100 - owner_occ_by_zip['Purchase Down %']
owner_occ_by_zip['mtg tax as % of med income'] = owner_occ_by_zip['total_tax_paid']/owner_occ_by_zip['Per Capita Income'] *100
owner_occ_by_zip['sales tax as % of med income'] = owner_occ_by_zip['New Sales Tax Amount']/owner_occ_by_zip['Per Capita Income'] *100
owner_occ_by_zip.head(2)

In [ ]:
investor_by_zip["total_tax_paid"] = investor_by_zip["Mtg Tx Pd"] + investor_by_zip["Mansion Tax Pd"]
investor_by_zip['percent borrowed'] = 100 - investor_by_zip['Purchase Down %']
investor_by_zip['mtg tax as % of med income'] = investor_by_zip['total_tax_paid']/investor_by_zip['Per Capita Income'] *100
investor_by_zip['sales tax as % of med income'] = investor_by_zip['New Sales Tax Amount']/investor_by_zip['Per Capita Income'] *100
investor_by_zip.head(2)

In [ ]:
investor_by_zip.dtypes

In [ ]:
zipcode_list = investor_by_zip.index.values.astype("str")
zipcode_list

## zipcodes of Manhattan: 10013, 10007, 10118, 10282, 10453, 11355, 10457, 11368
## The rest of zipcodes are in Brooklyn.

In [ ]:

manhattan_investor_df = investor_by_zip.loc[[10013, 10007, 10282, 10453, 11355, 10457, 11368], :].sort_values("Purchase Amt", ascending=False)
print(manhattan_df.shape)
manhattan_investor_df.head(8)

In [ ]:
manhattan_oocc_df = owner_occ_by_zip.loc[[10013, 10007, 10282, 10453, 11355, 10457, 11368], :].sort_values("Purchase Amt", ascending=False)
print(manhattan_oocc_df.shape)
manhattan_oocc_df.head(8)

In [ ]:
bklyn_investor_df = investor_by_zip.drop([10013, 10007, 10282, 10453, 11355, 10457, 11368], axis = 0)
bklyn_investor_df = bklyn_investor_df.sort_values("Purchase Amt", ascending=False)
print(bklyn_investor_df.shape)
bklyn_investor_df.head(8)

In [ ]:
bklyn_oocc_df = owner_occ_by_zip.drop([10013, 10007, 10282, 10453, 11355, 10457, 11368], axis = 0)
bklyn_oocc_df = bklyn_oocc_df.sort_values("Purchase Amt", ascending=False)
print(bklyn_oocc_df.shape)
bklyn_oocc_df.head()

In [ ]:

bklyn_oocc_df.set_index('Per Capita Income', inplace = True)
#bklyn_oocc_df['Zip'] = bklyn_oocc_df.index.astype(str)

bklyn_oocc_df.head(20)

In [ ]:
#make a chart of the % borrowed for the zip in ascending order but first sort dfs in order

bklyn_oocc_df.set_index('Per Capita Income', inplace = True)
bklyn_investor_df.set_index('Per Capita Income', inplace = True)
bklyn_oocc_df['perCI'] = bklyn_oocc_df.index.astype(str)
manhattan_oocc_df.set_index('Per Capita Income', inplace = True)
manhattan_investor_df.set_index('Per Capita Income', inplace = True)


plt.figure(figsize=(15,5))



plt.plot(bklyn_oocc_df['perCI'], bklyn_oocc_df['percent borrowed'], marker="x", label='Bklyn Residence Purchase')
plt.plot(bklyn_oocc_df['perCI'], bklyn_investor_df['percent borrowed'], marker="o", label='Bklyn Investment Purchase')
#plt.plot(manhatt_oocc_df['Per Capita Income'], investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
#plt.xticks(bklyn_oocc_df['Per Capita Income'], rotation=90)
plt.xticks(range(len(bklyn_oocc_df['Zip'])), fontsize=15, rotation='vertical')
plt.yticks(fontsize=15)
plt.xlabel('Zipcode sorted by Per Capita Income (in thousands)', fontsize=20)
plt.ylabel('Percent Borrowed', fontsize=20)
plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(10,5))

plt.plot(manhattan_oocc_df['Zip'], manhattan_oocc_df['percent borrowed'], marker="x", label='manhattan Residence Purchase')
plt.plot(manhattan_investor_df['Zip'], manhattan_investor_df['percent borrowed'], marker="o", label='manhattan Investment Purchase')
plt.xticks(fontsize=15, rotation='vertical')
plt.yticks(fontsize=15)
plt.xlabel('Zipcode sorted by Per Capita Income', fontsize=20)
plt.ylabel('Percent Borrowed', fontsize=20)
plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
plt.legend()
plt.show()

In [ ]:
#make a chart of the % borrowed for the zip in ascending order but first sort dfs in order
bklyn_oocc_df.sort_values(by =['Per Capita Income'], inplace=True)
bklyn_oocc_df['Zip'] = bklyn_oocc_df.index.astype(str)
bklyn_investor_df.sort_values(by =['Per Capita Income'], inplace=True)
bklyn_investor_df['Zip'] = bklyn_investor_df.index.astype(str)
manhattan_oocc_df.sort_values(by =['Per Capita Income'], inplace=True)
manhattan_oocc_df['Zip'] = manhattan_oocc_df.index.astype(str)
manhattan_investor_df.sort_values(by =['Per Capita Income'], inplace=True)
manhattan_investor_df['Zip'] = manhattan_investor_df.index.astype(str)

manhattan_investor_df.head(20)

plt.figure(figsize=(15,5))

plt.plot(bklyn_oocc_df['Zip'], bklyn_oocc_df['mtg tax as % of med income'], marker="x", label='Mtg tax as % of med income')
plt.plot(bklyn_investor_df['Zip'], bklyn_investor_df['percent borrowed'], marker="o", label='Sales tax as % of med income')
#plt.plot(manhatt_oocc_df['Per Capita Income'], investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
#plt.xticks(bklyn_oocc_df['Per Capita Income'], rotation=90)
plt.xticks(fontsize=15, rotation='vertical')
plt.yticks(fontsize=15)
plt.xlabel('Zipcode sorted by Per Capita Income', fontsize=20)
plt.ylabel('Percent Borrowed', fontsize=20)
plt.title('Average Income vs. Average Percent Borrowed across Zipcodes in Brooklyn 2022', fontsize=20)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,5))

plt.plot(zipcode_list, investor_by_zip["Per Capita Income"], marker="o", label='Per Capita Income')
plt.plot(zipcode_list, investor_by_zip["Purchase Amt"], marker="o", label='Purchase Amountt')
plt.plot(zipcode_list, investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
plt.xticks(zipcode_list, rotation=90)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Zipcode', fontsize=20)
plt.ylabel('Purchase Amount (in thousands)', fontsize=20)
plt.title('Average Income v.s. Average Purchase Amount across Zipcodes in NYC', fontsize=20)
plt.legend()
plt.show()

## Generate Manhattan dataframe

In [ ]:
mindex_list = manhattan_df.index.values

for zipcode in mindex_list:
    manhattan_df.rename(index={zipcode: str(zipcode)}, inplace=True)

manhattan_df.index.name = "Manhattan Zipcode"

mindex_list = manhattan_df.index.values 
mindex_list

In [ ]:
species = mindex_list

housing_data = {
    'Purchase Amt': manhattan_df["Purchase Amt"].values,
    'Amount Mortgaged': manhattan_df["Amount Mortgaged"].values,
    'Per Capita Income': manhattan_df["Per Capita Income"].values,
}

x = np.arange(len(species))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15,6))

for attribute, measurement in housing_data.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    #ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Dollar Amount', fontsize=13)
ax.set_title('Purchase & Mortgage (in thousands) Per Capita Income (in thousands)', fontsize=13)
ax.set_xticks(x + width, species)
ax.set_xlabel("Manhattan Zipcode", fontsize=13)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 500)
plt.suptitle('Purchase Amount v.s. Amount Mortgaged v.s. Per Capita Income in Manhattan', fontsize=15)
plt.show()

In [ ]:
fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4

manhattan_df["Per Capita Income"].plot(kind='bar', color='Orange', ax=ax, width=width, position=1)
manhattan_df["total_tax_paid"].plot(kind='bar', color='green', ax=ax2, width=width, position=0)

ax.set_ylabel('Per Capita Income in Thousands')
ax2.set_ylabel('Total Taxes paid')
ax.set_title('Scale to the left for Income and to the right for Taxes (in thousands)')
plt.suptitle('Per Capita Income v.s. Total Taxes paid',fontsize=15, y=1)
ax.legend()
ax2.legend(loc=9)

plt.show()

## Generate Broolyn dataframe

In [ ]:
brooklyn_df = investor_by_zip.drop([10013, 10007, 10282, 10453, 11355, 10457, 11368], axis=0)\
                            .sort_values("Purchase Amt", ascending=False).reset_index()
print(brooklyn_df.shape)

top5_brooklyn_df = brooklyn_df.loc[0:4]

top5_brooklyn_df.set_index(["Zipcode"], inplace=True)
top5_brooklyn_df

In [ ]:
# index_list = top5_brooklyn_df.index.values

# for zipcode in index_list:
#     top5_brooklyn_df.rename(index={zipcode: str(zipcode)}, inplace=True)

# top5_brooklyn_df.index.name = "Brooklyn Top 5 Zipcode"

index_list = top5_brooklyn_df.index.values.astype("str")
index_list

In [ ]:
species = index_list

housing_data = {
    'Purchase Amt': top5_brooklyn_df["Purchase Amt"].values,
    'Amount Mortgaged': top5_brooklyn_df["Amount Mortgaged"].values,
    'Per Capita Income': top5_brooklyn_df["Per Capita Income"].values,
}

x = np.arange(len(species))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15,6))

for attribute, measurement in housing_data.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    #ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Dollar Amount', fontsize=13)
ax.set_title('Purchase & Mortgage (in thousands) Per Capita Income (in thousands)', fontsize=13)
ax.set_xticks(x + width, species)
ax.set_xlabel("Brooklyn Top 5 Zipcode", fontsize=13)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 250)
plt.suptitle('Top 5 Purchase Amount v.s. Amount Mortgaged v.s. Per Capita Income in Brooklyn', fontsize=15)
plt.show()

## Line Chart Brooklyn - showing the amount borrowed by income level

In [ ]:
complete_df.shape

In [ ]:
brooklyn_complete = complete_df.drop([10013, 10007, 10282, 10453, 11355, 10457, 11368], axis=0)
brooklyn_complete.shape

In [ ]:
brooklyn_by_zip = brooklyn_complete.groupby(['Zipcode']).agg({'Count': 'sum', 'Purchase Amt': 'mean', 
                                                            'Purchase Down %': 'mean',
                                                            'Amount Mortgaged': 'mean', 
                                                            'Mtg Tx Pd': 'mean', 'Mansion Tax Pd': 'mean', 
                                                            'New Sales Tax Amount': 'mean', 
                                                            'Per Capita Income': 'mean'})
brooklyn_by_zip.shape

In [ ]:
brooklyn_by_zip_income_level = brooklyn_by_zip.sort_values("Per Capita Income")
brooklyn_by_zip_income_level.head()

In [ ]:
brooklyn_by_zip_income_level['Zipcode'] = brooklyn_by_zip_income_level.index
final_brooklyn_by_zip_income_level = brooklyn_by_zip_income_level.set_index("Per Capita Income")
final_brooklyn_by_zip_income_level.head()

In [ ]:
investor_df = complete_df[(complete_df['Owner Occ?'] ==1)]
investor_df.shape

In [ ]:
investor_index = investor_df.drop([10013, 10007, 10282, 10453, 11355, 10457, 11368], axis=0)
investor_index.shape

In [ ]:
investor_by_zip = investor_index.groupby(['Zipcode']).agg({'Count': 'sum', 'Purchase Amt': 'mean', 
                                                            'Purchase Down %': 'mean',
                                                            'Amount Mortgaged': 'mean', 
                                                            'Mtg Tx Pd': 'mean', 'Mansion Tax Pd': 'mean', 
                                                            'New Sales Tax Amount': 'mean', 
                                                            'Per Capita Income': 'mean'})
investor_by_zip.shape

In [ ]:
investor_income_level = investor_by_zip.sort_values("Per Capita Income")
investor_income_level.head()

In [ ]:
home_owner_df = complete_df[(complete_df['Owner Occ?'] ==0)]
home_owner_df.shape

In [ ]:
home_owner_df = complete_df[(complete_df['Owner Occ?'] ==0)]
home_owner_df.shape 

In [ ]:
home_owner_df = complete_df[(complete_df['Owner Occ?'] ==0)]
zip_codes_to_drop = [10013, 10007, 10282, 10453, 11355, 10457, 11368]
home_owner_index = home_owner_df[~home_owner_df['Zipcode'].isin(zip_codes_to_drop)]
home_owner_index.shape

In [ ]:
home_owner_by_zip = home_owner_index.groupby(['Zipcode']).agg({'Count': 'sum', 'Purchase Amt': 'mean', 
                                                             'Purchase Down %': 'mean',
                                                             'Amount Mortgaged': 'mean', 
                                                             'Mtg Tx Pd': 'mean', 'Mansion Tax Pd': 'mean', 
                                                             'New Sales Tax Amount': 'mean', 
                                                             'Per Capita Income': 'mean'})
home_owner_by_zip.shape

In [ ]:
home_owner_by_zip_income_level = home_owner_by_zip.sort_values("Per Capita Income")
home_owner_by_zip_income_level.head()

In [ ]:
columns_to_select = ['Column1', 'Column2', 'Column3']

In [ ]:
plt.figure(figsize=(20,5))

x_axis = brooklyn_by_zip_income_level['Zipcode'] 
y_axis = brooklyn_by_zip_income_level['Amount Mortgaged']
plt.bar(x_axis, y_axis, color='r', alpha=0.5, align="center")
# plt.plot(brooklyn_by_zip_income_level['Zipcode'], brooklyn_by_zip_income_level['Amount Mortgaged'], marker="o", label='Total Brooklyn Populaiton')
# plt.plot(zipcode_list, investor_by_zip["Purchase Amt"], marker="o", label='Purchase Amountt')
# plt.plot(zipcode_list, investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')

plt.xticks(x_axis, rotation=90)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Zipcode (Income=Lowest to Highest)', fontsize=20)
plt.ylabel('Purchase Amount (in thousands)', fontsize=20)
plt.title('Amount Mrtgaged vs Average Income Level by Zipcode', fontsize=20)
plt.legend()
plt.show()


In [ ]:
# plt.figure(figsize=(20,5))

# plt.plot(zipcode_list, investor_by_zip["Per Capita Income"], marker="o", label='Per Capita Income')
# plt.plot(zipcode_list, investor_by_zip["Purchase Amt"], marker="o", label='Purchase Amountt')
# plt.plot(zipcode_list, investor_by_zip["total_tax_paid"], marker="o", label='Total Tax Paid')
# plt.xticks(zipcode_list, rotation=90)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.xlabel('Zipcode', fontsize=20)
# plt.ylabel('Purchase Amount (in thousands)', fontsize=20)
# plt.title('Average Income v.s. Average Purchase Amount across Zipcodes in NYC', fontsize=20)
# plt.legend()
# plt.show()